**Подготовка к обучению моделей**

Импорт необходимых библиотек

In [ ]:
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import roc_auc_score

Загрузка данных и отбор признаков

In [ ]:
# загрузка датасета
dataframe = pd.read_csv('data/dataset.csv')

# категориальные признаки
categorical_features = dataframe.nunique()[3:].index.to_list()
# числовые признаки
numeric_features = dataframe.nunique()[:2].index.to_list()

x = dataframe.drop('target', axis=1)
y = dataframe['target']

random_state = 77
#  разделяем  выборку на тестовую (20%), обучающую (60%) и валидационную (20%)
x_train_val, x_test, y_train_val, y_test = train_test_split(
    x, y,
    test_size=0.2,
    stratify=y,
    random_state=random_state
)

x_train, x_val, y_train, y_val = train_test_split(
    x_train_val, y_train_val,
    test_size=0.25,  # четверть обучающей выборки
    stratify=y_train_val,
    random_state=random_state
)

**Базовая модель (Baseline)**

В качестве базовой модели мы взяли логистическую регрессию. Будем использовать прямое кодирование (One Hot Encoding) для категориальных признаков и стандартизацию (Standard Scaler) для числовых

In [ ]:
model = Pipeline([
    ('data_preprocessing', ColumnTransformer(
        transformers=[
            ('numeric', StandardScaler(), numeric_features),
            ('categorical', OneHotEncoder(), categorical_features)
        ])
     ),
    ('classification', LogisticRegression(
        class_weight='balanced',
        random_state=random_state
    ))
])

Обучение модели

In [ ]:
model.fit(x_train_val, y_train_val)

Оценка модели (ROC-AUC)

In [ ]:
y_pred = model.predict(x_test)
roc_auc = roc_auc_score(y_test, y_pred)
print(f"Значение метрики ROC-AUC: {roc_auc:.4f}")